In [201]:
import os
path = "icc.grammar"

grammar = ""
with open(path) as grammar_file:
    for line in grammar_file:
        grammar += line

print(grammar)


start: instruction+


instruction: VARNAME
            | domain_definition
            | type_definitions
            | predicate_definitions
            | action_definitions
            | VARIABLE

// domain definition
domain_definition: "(" "domain" VARNAME ")"

// types definition group
type_definitions: "(" ":" "type" type_definition+ ")"
type_definition: object_type_name "-" parent_type_name | object_type_name
object_type_name: VARNAME
parent_type_name: object_type_name | vector_type_name

// predicate definition group
predicate_definitions: "(" ":" "predicate" predicate_definition+ ")"
predicate_definition: predicate_name parameters
predicate_name: VARNAME
parameters: parameter+
parameter: VARIABLE

// action definition group
action_definitions: "(" ":" "action-definitions" action_definition+ ")"
action_definition: "(" "action" ":" action_name "parameters" ":" parameters "precondition" ":" precondition "effect" ":" effect ")"
action_name: VARNAME
precondition: VARNAME
effect: VAR

In [204]:
from lark import Lark

grammar = ""
with open(path) as grammar_file:
    for line in grammar_file:
        grammar += line
parser = Lark(grammar)

text = """
(domain blockworld)
(:type
    source - int
    block
    pos - vector[float, 3, 7] ;; the pos vector representation
    category - vector[int, 4] ;; the category vector representation
)
(:predicate
    clear ?x ?y
    is-red ?x
    is-blue ?x
    is-green ?x
)
(:action-definitions
    (
        action: pickup
        parameters: ?x ?y
        precondition:and
        effect:and
    )
    (
        action: placeon
        parameters: ?x ?y
        precondition: holdingxandyisclear
        effect: placesomethingonit
    )
)
"""

result = parser.parse(text)

class Domain:
    def __init__(self):
        self.domain_name = None # domain name indication

        self.types = {} # types as a diction, map from the type name to the actual type object
    
    def define_type(self, type_name, parent_name = None):
        if parent_name is None: parent_name = "object"
        self.types[type_name] = parent_name
    
    def define_action(self, action_name, parameters, precondition, effect):
        print("define")

    def load_domain_string(self, domain_string):
        pass
    
    def print_summary(self):
        print(f"\ndomain:\n  {self.domain_name}")
        print("types:")
        for key in self.types:
            print(f"  {key} - {self.types[key]}")
        print("actions:")

from lark import Transformer

class ICCTransformer(Transformer):
    def __init__(self):
        super().__init__()
        self.types = None
        self.domain = Domain()

    def domain_definition(self, domain_name):
        domain_name = domain_name[0]
        self.domain.domain_name = str(domain_name)
    
    """
    predicate definition handle part
    """
    def predicate_definition(self, args):
        predicate_name = args[0]
        parameters = args[1]
    
    def predicate_name(self, args):return str(args[0])

    def parameters(self, args):return [str(arg) for arg in args]
    
    def parameter(self, args):return str(args[0])
    
    def object_type_name(self, args):return str(args[0])
    
    """
    type definition handler part, use the args in the format of ?x ?y...
    """
    def type_definition(self, args):
        type_name = args[0]
        if len(args) == 1:self.domain.define_type(type_name)
        if len(args) == 2:self.domain.define_type(type_name, args[1])
    
    def parent_type_name(self, args):
        return args[0]
    
    def vector_type_name(self, args):
        vector_choice = args[0]
        vector_size = [str(num) for num in args[1:]]
        return f"vector[{vector_choice},{vector_size}]"
    
    def vector_choice(self, args):return args[0]
    
    def vector_size(self, args):return args[0]
    
    def number(self,args):return str(args[0])

    """
    action definition handler part, use the format of
    (
        action: name
        parameters: ?x ?y ...
        precondition: (and)
        effect: (and)
    )
    """
    def action_definition(self, args):
        #print(args)
        self.domain.define_action()
    
    def action_name(self, args): return str(args[0])

    def precondition(self, args): return str(args[0])

    def effect(self, args): return str(args[0])
    
    
class Precondition:
    def __init__(self, bool_expression):
        self.bool_expression = bool_expression

class Effect:
    def __init__(self, bool_expression):
        self.bool_expression = bool_expression

icc_transformer = ICCTransformer()

#print(result.pretty())

icc_transformer.transform(result)

icc_transformer.domain.print_summary()


domain:
  blockworld
types:
  source - int
  block - object
  pos - vector[float,['3', '7']]
  category - vector[int,['4']]
actions:
